In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
trn = pd.read_csv('./data/train.csv')
tst = pd.read_csv('./data/test.csv')

In [3]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [4]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

## 결측치 처리

- Origin_Airport, Origin_Airport_ID : 1:1 대응 확인
- Airport, States 관계 확인
- 결측 대체

In [5]:
# ## 출발 공항, 공항번호 1:1 대응 확인
# origin_airports = trn['Origin_Airport'].unique().tolist()

# check = []
# for airport in origin_airports:
#     id_list = trn[trn['Origin_Airport']==airport]['Origin_Airport_ID'].unique().tolist()
#     if (len(id_list) != 1):
#         check.append(airport)
# print(check)

# ## 도착 공항, 공항번호 1:1 대응 확인
# depart_airports = trn['Destination_Airport'].unique().tolist()
# check = []
# for airport in depart_airports:
#     id_list = trn[trn['Destination_Airport']==airport]['Destination_Airport_ID'].unique().tolist()
#     if (len(id_list) != 1):
#         check.append(airport)
# print(check)

In [6]:
## origin state - airport dictionary 생성
temp = trn[~trn['Origin_State'].isnull()]
states = temp['Origin_State'].unique().tolist()
print(len(states))

origin_state_airpot_dict = dict()
for stat in states:
    airports = temp[temp['Origin_State']==stat]['Origin_Airport'].unique().tolist()
    for air in airports:
        origin_state_airpot_dict[air] = stat
        

temp = trn[~trn['Destination_State'].isnull()]
destination_states = temp['Destination_State'].unique().tolist()
print(len(destination_states))

destination_state_airpot_dict = dict()
for stat in destination_states:
    airports = temp[temp['Destination_State']==stat]['Destination_Airport'].unique().tolist()
    for air in airports:
        destination_state_airpot_dict[air] = stat

## ohio yng 공항은 데이터 내에서 찾을 수 없음 -> 검색정보 수기로 채우기
destination_state_airpot_dict['YNG'] = 'Ohio'

52
52


- dictionary 정보로 states 결측 대체
    - origin_state_airpot_dict
    - destination_states

In [7]:
def isna(x):
    return x!=x

In [8]:
trn['Origin_State'] = trn.apply(lambda x:origin_state_airpot_dict[x['Origin_Airport']] if(isna(x['Origin_State'])==True) else x['Origin_State'], axis=1)
trn['Destination_State'] = trn.apply(lambda x:destination_state_airpot_dict[x['Destination_Airport']] if(isna(x['Destination_State'])==True) else x['Destination_State'], axis=1)

In [9]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

=> State 결측 제거 완료

- Airline 정보확인

In [10]:
## Airline, Carrier_ID(DOT) 1:1 대응 확인
temp = trn[~trn['Airline'].isnull()]
temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)
airlines = temp.Airline.unique().tolist()

dot_dict = dict()
check = []
cnt=0
for air in airlines:
    dot = temp[temp['Airline']==air]['Carrier_ID(DOT)'].unique().tolist()    
    dot = [x for x in dot if x != 'nan']
    cnt+=1
    if (len(dot) != 1):
        check.append(air)
        break
    dot_dict[dot[0]] = air
print(check)



## IATA 정보 airline과 1:1 대응인 지점 확인
temp = trn[~trn['Carrier_Code(IATA)'].isnull()]
iata_ids = temp['Carrier_Code(IATA)'].unique().tolist()

single_iata_dict = dict()
for iata in iata_ids:
    airline = temp[temp['Carrier_Code(IATA)']==iata].Airline.unique()
    airline = [x for x in airline if isna(x) == False]
    if(len(airline)==1):
        single_iata_dict[iata] = airline[0]
        
        
## Airline,tail_number 확인
temp = trn[~trn['Airline'].isnull()]
airlines = temp.Airline.unique().tolist()
airline_tail_dict = dict()

for air in airlines:
    tail_list = temp[temp['Airline']==air]['Tail_Number'].unique().tolist()
    airline_tail_dict[air] = tail_list

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_46113/2898317092.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)


[]


- airline 결측 대체
    - dot dictionary : airline과 1:1 대응
    - single_iata_dict : 일부 iata정보 airline과 1:1 대응
    - airline_tail_dict : 일부 airline과 tail_number 1:1 대응

In [11]:
trn['Carrier_ID(DOT)'] = trn['Carrier_ID(DOT)'].astype(str)
for i in range(len(trn)):
    if(isna(trn.loc[i, 'Airline']) == True):
        if(trn.loc[i, 'Carrier_ID(DOT)'] != 'nan'):
            trn.loc[i, 'Airline'] = dot_dict[trn.loc[i, 'Carrier_ID(DOT)']]
            
        elif(trn.loc[i, 'Carrier_Code(IATA)'] in single_iata_dict):
            trn.loc[i, 'Airline'] = single_iata_dict[trn.loc[i, 'Carrier_Code(IATA)']]
            
        single_tail_check = []
        for air in airlines:
            if (trn.loc[i, 'Tail_Number'] in airline_tail_dict[air]):
                single_tail_check.append(air)
        ## 1:1 대응의 경우 airline 대체
        if (len(single_tail_check) == 1):
            trn.loc[i, 'Airline'] = single_tail_check[0]

In [12]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                        315
Carrier_Code(IATA)          108990
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

=> 항공사 정보가 중복되는 315건 => 동일한 tail number에서 빈도수 많은 값으로 대체

In [13]:
for i in range(len(trn)):
    if(isna(trn.loc[i, 'Airline']) == True):
        try:
            trn.loc[i, 'Airline'] = list(dict(trn[trn['Tail_Number']==trn.loc[i, 'Tail_Number']]['Airline'].value_counts()[:1]).keys())[0]
        except:
            pass

In [14]:
# 남은 2개는 정황상 Delta Air Lines Inc.(같은 항공편 조회)
trn['Airline'] = trn['Airline'].fillna('Delta Air Lines Inc.')

In [15]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)          108990
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
dtype: int64

- 출발 시간, 착륙시간 확인

In [16]:
## 시간, 분 분리
trn['Estimated_Departure_Hour'] = trn['Estimated_Departure_Time'].apply(lambda x:x//100)
trn['Estimated_Departure_Min'] = trn['Estimated_Departure_Time'].apply(lambda x:x%100)
trn['Estimated_Arrival_Hour'] = trn['Estimated_Arrival_Time'].apply(lambda x:x//100)
trn['Estimated_Arrival_Min'] = trn['Estimated_Arrival_Time'].apply(lambda x:x%100)

In [17]:
## 항공편별 출발, 도착 시간 평균
matching_df = round(trn.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())
matching_df['identity'] = matching_df['Origin_Airport']+matching_df['Destination_Airport']
matching_df.head(3)

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_46113/968825694.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  matching_df = round(trn.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())


,Origin_Airport,Destination_Airport,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min,identity
0,ABE,ATL,12.0,29.0,14.0,16.0,ABEATL
1,ABE,BNA,17.0,37.0,18.0,33.0,ABEBNA
2,ABE,CLT,11.0,30.0,13.0,34.0,ABECLT


In [18]:
trn['identity'] = trn['Origin_Airport'] + trn['Destination_Airport']

## 둘다 비어있는경우
temp = trn[(trn['Estimated_Departure_Time'].isnull())&(trn['Estimated_Arrival_Time'].isnull())]
ids = temp['ID'].tolist()
trn = trn[~trn['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Departure_Hour', 'Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min']), matching_df.iloc[:,2:], left_on='identity', right_on='identity')
trn = pd.concat([trn,temp])

## 남은 것중 출발 비어있는것
temp = trn[trn['Estimated_Departure_Time'].isnull()]
ids = temp['ID'].tolist()
trn = trn[~trn['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Departure_Hour', 'Estimated_Departure_Min']), matching_df[['Estimated_Departure_Hour', 'Estimated_Departure_Min','identity']], left_on='identity', right_on='identity')
trn = pd.concat([trn,temp])

## 남은 것중 도착 비어있는것
temp = trn[trn['Estimated_Arrival_Time'].isnull()]
ids = temp['ID'].tolist()
trn = trn[~trn['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Arrival_Hour', 'Estimated_Arrival_Min']), matching_df[['Estimated_Arrival_Hour', 'Estimated_Arrival_Min','identity']], left_on='identity', right_on='identity')
trn = pd.concat([trn,temp])

## 참고할 항공편 없는 경우 일단 평균대체
trn['Estimated_Departure_Hour'] = trn['Estimated_Departure_Hour'].fillna(round(trn['Estimated_Departure_Hour'].mean()))
trn['Estimated_Departure_Min'] = trn['Estimated_Departure_Min'].fillna(round(trn['Estimated_Departure_Min'].mean()))
trn['Estimated_Arrival_Hour'] = trn['Estimated_Arrival_Hour'].fillna(round(trn['Estimated_Arrival_Hour'].mean()))
trn['Estimated_Arrival_Min'] = trn['Estimated_Arrival_Min'].fillna(round(trn['Estimated_Arrival_Min'].mean()))

In [19]:
trn.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                          0
Carrier_Code(IATA)          108990
Carrier_ID(DOT)                  0
Tail_Number                      0
Delay                       744999
Estimated_Departure_Hour         0
Estimated_Departure_Min          0
Estimated_Arrival_Hour           0
Estimated_Arrival_Min            0
identity                         0
dtype: int64

In [21]:
## 정렬
trn = trn.sort_values(by=['ID']).reset_index(drop=True)
trn

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,...,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min,identity
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,...,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,13.0,33.0,14.0,24.0,OKCHOU
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,...,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,7.0,40.0,10.0,24.0,ORDSLC
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,...,American Airlines Inc.,AA,19805.0,N103US,NaN,16.0,10.0,18.0,5.0,CLTLGA
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,...,United Air Lines Inc.,UA,nan,N595UA,NaN,9.0,5.0,17.0,35.0,LAXEWR
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,...,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,9.0,0.0,10.0,19.0,SFOACV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,Illinois,...,United Air Lines Inc.,UA,19977.0,N477UA,NaN,9.0,36.0,12.0,43.0,ORDPHL
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,North Dakota,...,SkyWest Airlines Inc.,DL,nan,N439SW,NaN,9.0,20.0,10.0,28.0,FARMSP
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,California,...,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,8.0,0.0,13.0,40.0,OAKHOU
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,...,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,16.0,13.0,18.0,24.0,BNAATL


- 필요없는 column drop
    - ID : 식별자
    - Origin_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
    - Destination_Airport_ID: 해당 항공편 출발 공항의 고유 ID (US DOT ID)
    - Cancelled : 전부 0
    - Diverted : 전부 0
    - Carrier_Code(IATA), Carrier_ID(DOT) : 항공사 정보로 통일
    - Estimated_Departure_Time, Estimated_Arrival_Time : 시간정보 추출완료
    - identity : 전처리중 생긴 정보

In [23]:
trn = trn.drop(columns=['ID','Origin_Airport_ID','Destination_Airport_ID','Cancelled','Diverted','Carrier_Code(IATA)','Carrier_ID(DOT)','Estimated_Departure_Time','Estimated_Arrival_Time','identity'])

In [27]:
trn.to_csv('./data/train_fillna.csv',index=False)

### test set

In [29]:
## origin state - airport dictionary 생성
temp = tst[~tst['Origin_State'].isnull()]
states = temp['Origin_State'].unique().tolist()
print(len(states))

origin_state_airpot_dict = dict()
for stat in states:
    airports = temp[temp['Origin_State']==stat]['Origin_Airport'].unique().tolist()
    for air in airports:
        origin_state_airpot_dict[air] = stat
        

temp = tst[~tst['Destination_State'].isnull()]
destination_states = temp['Destination_State'].unique().tolist()
print(len(destination_states))

destination_state_airpot_dict = dict()
for stat in destination_states:
    airports = temp[temp['Destination_State']==stat]['Destination_Airport'].unique().tolist()
    for air in airports:
        destination_state_airpot_dict[air] = stat


52
52


In [48]:
## 추가 공항 정보 입력
origin_state_airpot_dict['RIW'] = 'Wyoming'
origin_state_airpot_dict['SHR'] = 'Wyoming'
origin_state_airpot_dict['DDC'] = 'Kansas'
origin_state_airpot_dict['DEC'] = 'Illinois'

destination_state_airpot_dict['YNG'] = 'Ohio'
destination_state_airpot_dict['RIW'] = 'Wyoming'
destination_state_airpot_dict['SHR'] = 'Wyoming'
destination_state_airpot_dict['DDC'] = 'Kansas'
destination_state_airpot_dict['DEC'] = 'Illinois'

In [49]:
tst['Origin_State'] = tst.apply(lambda x:origin_state_airpot_dict[x['Origin_Airport']] if(isna(x['Origin_State'])==True) else x['Origin_State'], axis=1)
tst['Destination_State'] = tst.apply(lambda x:destination_state_airpot_dict[x['Destination_Airport']] if(isna(x['Destination_State'])==True) else x['Destination_State'], axis=1)

In [50]:
## Airline, Carrier_ID(DOT) 1:1 대응 확인
temp = tst[~tst['Airline'].isnull()]
temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)
airlines = temp.Airline.unique().tolist()

dot_dict = dict()
check = []
cnt=0
for air in airlines:
    dot = temp[temp['Airline']==air]['Carrier_ID(DOT)'].unique().tolist()    
    dot = [x for x in dot if x != 'nan']
    cnt+=1
    if (len(dot) != 1):
        check.append(air)
        break
    dot_dict[dot[0]] = air
print(check)



## IATA 정보 airline과 1:1 대응인 지점 확인
temp = tst[~tst['Carrier_Code(IATA)'].isnull()]
iata_ids = temp['Carrier_Code(IATA)'].unique().tolist()

single_iata_dict = dict()
for iata in iata_ids:
    airline = temp[temp['Carrier_Code(IATA)']==iata].Airline.unique()
    airline = [x for x in airline if isna(x) == False]
    if(len(airline)==1):
        single_iata_dict[iata] = airline[0]
        
        
## Airline,tail_number 확인
temp = tst[~tst['Airline'].isnull()]
airlines = temp.Airline.unique().tolist()
airline_tail_dict = dict()

for air in airlines:
    tail_list = temp[temp['Airline']==air]['Tail_Number'].unique().tolist()
    airline_tail_dict[air] = tail_list

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_46113/4149931972.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Carrier_ID(DOT)'] = temp['Carrier_ID(DOT)'].astype(str)


[]


In [51]:
tst['Carrier_ID(DOT)'] = tst['Carrier_ID(DOT)'].astype(str)
for i in range(len(tst)):
    if(isna(tst.loc[i, 'Airline']) == True):
        if(tst.loc[i, 'Carrier_ID(DOT)'] != 'nan'):
            tst.loc[i, 'Airline'] = dot_dict[tst.loc[i, 'Carrier_ID(DOT)']]
            
        elif(tst.loc[i, 'Carrier_Code(IATA)'] in single_iata_dict):
            tst.loc[i, 'Airline'] = single_iata_dict[tst.loc[i, 'Carrier_Code(IATA)']]
            
        single_tail_check = []
        for air in airlines:
            if (tst.loc[i, 'Tail_Number'] in airline_tail_dict[air]):
                single_tail_check.append(air)
        ## 1:1 대응의 경우 airline 대체
        if (len(single_tail_check) == 1):
            tst.loc[i, 'Airline'] = single_tail_check[0]

In [52]:
for i in range(len(tst)):
    if(isna(tst.loc[i, 'Airline']) == True):
        try:
            tst.loc[i, 'Airline'] = list(dict(tst[tst['Tail_Number']==tst.loc[i, 'Tail_Number']]['Airline'].value_counts()[:1]).keys())[0]
        except:
            pass

In [55]:
## 시간, 분 분리
tst['Estimated_Departure_Hour'] = tst['Estimated_Departure_Time'].apply(lambda x:x//100)
tst['Estimated_Departure_Min'] = tst['Estimated_Departure_Time'].apply(lambda x:x%100)
tst['Estimated_Arrival_Hour'] = tst['Estimated_Arrival_Time'].apply(lambda x:x//100)
tst['Estimated_Arrival_Min'] = tst['Estimated_Arrival_Time'].apply(lambda x:x%100)

In [56]:
## 항공편별 출발, 도착 시간 평균
matching_df = round(tst.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())
matching_df['identity'] = matching_df['Origin_Airport']+matching_df['Destination_Airport']
matching_df.head(3)

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_46113/967718487.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  matching_df = round(tst.groupby(['Origin_Airport','Destination_Airport'])['Estimated_Departure_Hour','Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min'].mean().reset_index())


,Origin_Airport,Destination_Airport,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min,identity
0,ABE,ATL,12.0,24.0,14.0,19.0,ABEATL
1,ABE,CLT,12.0,34.0,14.0,33.0,ABECLT
2,ABE,DTW,12.0,32.0,14.0,33.0,ABEDTW


In [57]:
tst['identity'] = tst['Origin_Airport'] + tst['Destination_Airport']

## 둘다 비어있는경우
temp = tst[(tst['Estimated_Departure_Time'].isnull())&(tst['Estimated_Arrival_Time'].isnull())]
ids = temp['ID'].tolist()
tst = tst[~tst['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Departure_Hour', 'Estimated_Departure_Min','Estimated_Arrival_Hour','Estimated_Arrival_Min']), matching_df.iloc[:,2:], left_on='identity', right_on='identity')
tst = pd.concat([tst,temp])

## 남은 것중 출발 비어있는것
temp = tst[tst['Estimated_Departure_Time'].isnull()]
ids = temp['ID'].tolist()
tst = tst[~tst['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Departure_Hour', 'Estimated_Departure_Min']), matching_df[['Estimated_Departure_Hour', 'Estimated_Departure_Min','identity']], left_on='identity', right_on='identity')
tst = pd.concat([tst,temp])

## 남은 것중 도착 비어있는것
temp = tst[tst['Estimated_Arrival_Time'].isnull()]
ids = temp['ID'].tolist()
tst = tst[~tst['ID'].isin(ids)]

temp = pd.merge(temp.drop(columns=['Estimated_Arrival_Hour', 'Estimated_Arrival_Min']), matching_df[['Estimated_Arrival_Hour', 'Estimated_Arrival_Min','identity']], left_on='identity', right_on='identity')
tst = pd.concat([tst,temp])

## 참고할 항공편 없는 경우 일단 평균대체
tst['Estimated_Departure_Hour'] = tst['Estimated_Departure_Hour'].fillna(round(tst['Estimated_Departure_Hour'].mean()))
tst['Estimated_Departure_Min'] = tst['Estimated_Departure_Min'].fillna(round(tst['Estimated_Departure_Min'].mean()))
tst['Estimated_Arrival_Hour'] = tst['Estimated_Arrival_Hour'].fillna(round(tst['Estimated_Arrival_Hour'].mean()))
tst['Estimated_Arrival_Min'] = tst['Estimated_Arrival_Min'].fillna(round(tst['Estimated_Arrival_Min'].mean()))

In [58]:
## 정렬
tst = tst.sort_values(by=['ID']).reset_index(drop=True)
tst

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,...,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Estimated_Departure_Hour,Estimated_Departure_Min,Estimated_Arrival_Hour,Estimated_Arrival_Min,identity
0,TEST_000000,12,16,1156.0,NaN,0,0,IAH,12266,Texas,...,191.0,United Air Lines Inc.,UA,nan,N79402,11.0,56.0,16.0,30.0,IAHSAT
1,TEST_000001,9,12,1500.0,1715.0,0,0,EWR,11618,New Jersey,...,746.0,Delta Air Lines Inc.,DL,19790.0,N3765,15.0,0.0,17.0,15.0,EWRATL
2,TEST_000002,3,6,1600.0,1915.0,0,0,ORD,13930,Illinois,...,733.0,United Air Lines Inc.,UA,19977.0,N413UA,16.0,0.0,19.0,15.0,ORDLGA
3,TEST_000003,5,18,1920.0,2045.0,0,0,OAK,13796,California,...,337.0,Southwest Airlines Co.,WN,19393.0,N905WN,19.0,20.0,20.0,45.0,OAKLAX
4,TEST_000004,7,7,1915.0,2152.0,0,0,FLL,11697,Florida,...,2343.0,JetBlue Airways,B6,20409.0,N945JT,19.0,15.0,21.0,52.0,FLLLAX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TEST_999995,6,2,NaN,2240.0,0,0,LAX,12892,California,...,862.0,Southwest Airlines Co.,WN,19393.0,N720WN,12.0,30.0,22.0,40.0,LAXDEN
999996,TEST_999996,6,8,600.0,648.0,0,0,BUF,10792,New York,...,473.0,United Air Lines Inc.,UA,19977.0,N401UA,6.0,0.0,6.0,48.0,BUFORD
999997,TEST_999997,6,11,1710.0,1907.0,0,0,IAD,12264,Virginia,...,383.0,Mesa Airlines Inc.,UA,20378.0,N510MJ,17.0,10.0,19.0,7.0,IADDTW
999998,TEST_999998,11,17,NaN,500.0,0,0,SAN,14679,California,...,2588.0,JetBlue Airways,B6,20409.0,N986JB,14.0,30.0,5.0,0.0,SANBOS


In [59]:
tst = tst.drop(columns=['ID','Origin_Airport_ID','Destination_Airport_ID','Cancelled','Diverted','Carrier_Code(IATA)','Carrier_ID(DOT)','Estimated_Departure_Time','Estimated_Arrival_Time','identity'])

In [60]:
tst.to_csv('./data/test_fillna.csv',index=False)